In [1]:
# data initialization
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("map") \
    .getOrCreate()
business = spark.read.option("header",True) \
.option("inferSchema", True) \
.format("json").load("shared/student-data/yelp/yelp_academic_dataset_business.json")
restaurant = business.filter((col('categories').contains('Restaurants')) |\
                      (col('categories').contains('restaurants')) |\
                      (col('categories').contains('Restaurant')) |\
                      (col('categories').contains('restaurant')) |\
                      (col('categories').contains('RESTAURANT')))

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/14 23:22:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/14 23:22:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
# get clean df
dataWithCoordinatesZipStars = restaurant.select('city','state', 'longitude', 'latitude', 'postal_code', 'stars')

In [4]:
dataWithCoordinatesZipStars.toPandas()

,city,state,longitude,latitude,postal_code,stars
0,Boulder,CO,-105.283348,40.017544,80302,4.0
1,Portland,OR,-122.593331,45.588906,97218,4.0
2,Vancouver,BC,-123.101333,49.251342,V5V,3.5
3,Columbus,OH,-82.997471,39.947007,43206,4.5
4,Peabody,MA,-70.973438,42.541155,01960,4.0
...,...,...,...,...,...,...
50788,Tigard,OR,-122.752535,45.394457,97224,2.0
50789,Bee Cave,TX,-97.937394,30.304761,78738,3.0
50790,Atlanta,GA,-84.367084,33.810996,30324,3.0
50791,Portland,OR,-122.685153,45.529647,97209,4.0


In [8]:
# get restaurant data from Atlanta
# same for Boston, Portland, Austin
dataWithCoordinatesZipStars.filter(col('city') == 'Atlanta').toPandas()

,city,state,longitude,latitude,postal_code,stars
0,Atlanta,GA,-84.363578,33.846335,30326,3.0
1,Atlanta,GA,-84.378539,33.842875,30305,4.0
2,Atlanta,GA,-84.280918,33.896640,30340,3.0
3,Atlanta,GA,-84.380375,33.772758,30308,4.0
4,Atlanta,GA,-84.425514,33.640879,30320,3.5
...,...,...,...,...,...,...
4175,Atlanta,GA,-84.362535,33.774230,30306,3.5
4176,Atlanta,GA,-84.384517,33.785008,30309,3.5
4177,Atlanta,GA,-84.359011,33.773536,30306,2.5
4178,Atlanta,GA,-84.365957,33.852144,30326,3.0


In [10]:
# export the data so that I can use it in javascript
dataWithCoordinatesZipStars.toPandas().to_csv('dataWithCoordinatesZipStars.csv')

In [5]:
# filter the restaurant with stars >= 4.5
usTopRated = restaurant.filter(col('stars') >= 4.5)

In [8]:
# only select the longitude and latitude data for map plotting
usTopRatedPoints = usTopRated.select('longitude','latitude') 

In [9]:
# export to csv
usTopRatedPoints.toPandas().to_csv('usTopRated.csv')

In [15]:
# filter out Vancouver before counintg since it is not a US city.
restaurant.filter(col('city') != 'Vancouver').groupBy('city').count().orderBy(col('count').desc()).toPandas()

,city,count
0,Portland,5737
1,Austin,4965
2,Atlanta,4180
3,Orlando,3749
4,Boston,2846
...,...,...
453,Quincy Center,1
454,Brookline Village,1
455,Polaris,1
456,NEW WESTMINSTER,1
